In [1]:
import numpy as np
import fitsio
from astropy.table import Table, join
import os, sys
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pandas as pd
import corner
import mpl_scatter_density

# Make the norm object to define the image stretch
from astropy.visualization import LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
norm = ImageNormalize(vmin=0., vmax=1000, stretch=LogStretch())

import sys
sys.path.insert(0, '/global/homes/s/severett/repos/balutils/')
from balutils import stacked_catalogs as sc

import seaborn as sb
plt.style.use('seaborn')
sb.set_context("notebook", font_scale=1.5)

In [2]:
def calc_gold_cuts(cat, flags_gold_colname='FLAGS_GOLD_SOF_ONLY'):
    gold_cuts = np.where( 
                          (cat['match_flag_1.5_asec'] < 2) &
                          (cat['flags_foreground'] == 0) &
                          (cat['flags_badregions'] < 2) &
                          (cat['flags_footprint'] == 1) &
                          (cat[flags_gold_colname] < 2)
                        )
    return gold_cuts

def calc_gold_photometry_cuts(cat, flags_gold_colname='FLAGS_GOLD_SOF_ONLY'):
    gold_phot_cuts = np.where( 
                          (cat['match_flag_1.5_asec'] < 2) &
                          (cat['flags_foreground'] == 0) &
                          (cat['flags_badregions'] < 2) &
                          (cat['flags_footprint'] == 1) &
                          (cat[flags_gold_colname] < 8)
                        )
    return gold_phot_cuts

def calc_shape_cuts(cat):
    shape_cuts = np.where( (cat['flags'] == 0) &
                           (cat['size_ratio'] > 0.5) &
                           (cat['snr'] > 10) &
                           (cat['snr'] < 1000)
                          )
    return shape_cuts

In [ ]:
run = 'y3-merged'
ver = '1.2'

match_file = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/balrog_matched_catalog_sof_{}_v{}.fits'.format(run, ver, run, ver)
det_file = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/balrog_detection_catalog_sof_{}_v{}.fits'.format(run, ver, run, ver)

match_cols = ['bal_id', 'meas_cm_mag_deredden',  'true_bdf_mag_deredden', 'true_gap_flux_fwhm4asec', 'meas_gapflux']
det_cols = ['bal_id', 'meas_FLAGS_GOLD_SOF_ONLY', 'flags_foreground', 'flags_footprint', 'flags_badregions', 'match_flag_1.5_asec']
# sof = sc.BalrogMatchedCatalog(match_file, det_file, match_cols=match_cols, match_type='sof_only', vb=True)

print('Loading matched file...')
match = Table(fitsio.read(match_file))
print('Loading det file...')
det = Table(fitsio.read(det_file))
print('joining...')
sof = join(match, det, keys='bal_id', join_type='inner')
assert len(match) == len(sof)

Loading matched file...


In [ ]:
gold_cuts = calc_gold_cuts(sof)
gold_photometry_cuts = calc_gold_photometry_cuts(sof)
shape_cuts = calc_shape_cuts(sof)

print('Making gold cuts...')
gold = sof[gold_cuts]
print('Writing...')
gold_outfile = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/joined_balrog_gold_sample.fits'.format(run, ver, run, ver)
gold.write(gold_outfile)

print('Making gold_photometry cuts...')
gold_phot = sof[gold_photometry_cuts]
print('Writing...')
gold_phot_outfile = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/joined_balrog_gold_photometry_sample.fits'.format(run, ver, run, ver)
gold_phot.write(gold_phot_outfile)